## Importing necessary libaries

In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.metrics import mean_squared_error, accuracy_score, recall_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier

## Load machine failure file

In [109]:
data = pd.read_csv('../machine failure.csv')
data

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,0,0,0,0,0
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,0,0,0,0,0
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,0,0,0,0,0
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,0,0,0,0,0


## Data cleaning

In [110]:
#Using getdummies to turn the Type (categorical) column into 3 binary columns
data = pd.get_dummies(data, columns=['Type', ])
data.head()

,UDI,Product ID,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Type_H,Type_L,Type_M
0,1,M14860,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,0,0,1
1,2,L47181,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,0,1,0
2,3,L47182,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,0,1,0
3,4,L47183,298.2,308.6,1433,39.5,7,0,0,0,0,0,0,0,1,0
4,5,L47184,298.2,308.7,1408,40.0,9,0,0,0,0,0,0,0,1,0


In [111]:
#Inspecting data types
data.dtypes

UDI                          int64
Product ID                  object
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
Machine failure              int64
TWF                          int64
HDF                          int64
PWF                          int64
OSF                          int64
RNF                          int64
Type_H                       uint8
Type_L                       uint8
Type_M                       uint8
dtype: object

## Consolidating types of machine failure into the machine failure column

In [112]:
data['Machine failure'] = np.where((data['TWF'] == 1) | (data['HDF'] == 1) | (data['PWF'] == 1) | (data['OSF'] == 1), 1, 0)

cleaned_data = data.drop(['UDI','Product ID', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1)

cleaned_data

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Type_H,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,0,0,0,1
1,298.2,308.7,1408,46.3,3,0,0,1,0
2,298.1,308.5,1498,49.4,5,0,0,1,0
3,298.2,308.6,1433,39.5,7,0,0,1,0
4,298.2,308.7,1408,40.0,9,0,0,1,0
...,...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,0,0,0,1
9996,298.9,308.4,1632,31.8,17,0,1,0,0
9997,299.0,308.6,1645,33.4,22,0,0,0,1
9998,299.0,308.7,1408,48.5,25,0,1,0,0


## Building Ridge regression model

In [113]:
#Load the predictor and target variables
X = cleaned_data.drop(columns=['Machine failure'])
y = cleaned_data['Machine failure']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize (scale) the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE resampling technique
smote = SMOTE()
X_train_scaled_and_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Create a RidgeCV model with alpha optimization using cross-validation
alphas = [0.01, 0.1, 1.0, 10.0]  # List of alpha values to test
ridge = RidgeCV(alphas=alphas, store_cv_values=True)

# Fit the Ridge regression model to the training data
ridge.fit(X_train_scaled_and_resampled, y_train_resampled)

# Print the best alpha determined by cross-validation
print("Best alpha:", ridge.alpha_)

# Evaluate the model on the test data
y_pred = ridge.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on Test Data:", mse)

# Turn the outputs into binary format
y_pred_binary = (y_pred > 0.5).astype(int)

#Calculate the recall score
recall = recall_score(y_test, y_pred_binary)
print(f"Recall Score: {recall:.4f}")

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy Score: {accuracy:.4f}", "\nDisclaimer: Accuracy is not a reliable metric for regressions")

Best alpha: 1.0
Mean Squared Error on Test Data: 0.12434412712474913
Recall Score: 0.8222
Accuracy Score: 0.8377 
Disclaimer: Accuracy is not a reliable metric for regressions


## Analyzing feature weights

In [114]:
feature_weights = ridge.coef_
feature_names = X.columns

print("Feature Weights (Coefficients):")
for feature, weight in zip(feature_names, feature_weights):
    print(f"{feature}: {weight:.4f}")

Feature Weights (Coefficients):
Air temperature [K]: 0.2995
Process temperature [K]: -0.2195
Rotational speed [rpm]: 0.2755
Torque [Nm]: 0.3793
Tool wear [min]: 0.1239
Type_H: -0.0223
Type_L: 0.0127
Type_M: 0.0011


## Building Lasso regression model

In [115]:
#Load the predictor and target variables
X = cleaned_data.drop(columns=['Machine failure'])
y = cleaned_data['Machine failure']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize (scale) the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE resampling technique
smote = SMOTE()
X_train_scaled_and_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Create a LassoCV model with alpha optimization using cross-validation
alphas = [0.01, 0.1, 1.0, 10.0]  # List of alpha values to test
lasso = LassoCV(alphas=alphas, cv=5)

# Fit the Lasso regression model to the training data
lasso.fit(X_train_scaled_and_resampled, y_train_resampled)

# Print the best alpha determined by cross-validation
print("Best alpha:", lasso.alpha_)

# Evaluate the model on the test data
y_pred = lasso.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on Test Data:", mse)

# Turn the outputs into binary format
y_pred_binary = (y_pred > 0.5).astype(int)

#Calculate the recall score
recall = recall_score(y_test, y_pred_binary)
print(f"Recall Score: {recall:.4f}")

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy Score: {accuracy:.4f}", "\nDisclaimer: Accuracy is not a reliable metric for regressions")

Best alpha: 0.01
Mean Squared Error on Test Data: 0.12255661665640931
Recall Score: 0.8333
Accuracy Score: 0.8407 
Disclaimer: Accuracy is not a reliable metric for regressions


## Analyzing feature weights

In [116]:
feature_weights = lasso.coef_
feature_names = X.columns

print("Feature Weights (Coefficients):")
for feature, weight in zip(feature_names, feature_weights):
    print(f"Feature {feature}: {weight:.4f}")

Feature Weights (Coefficients):
Feature Air temperature [K]: 0.2199
Feature Process temperature [K]: -0.1345
Feature Rotational speed [rpm]: 0.2491
Feature Torque [Nm]: 0.3521
Feature Tool wear [min]: 0.1123
Feature Type_H: -0.0153
Feature Type_L: 0.0089
Feature Type_M: -0.0000


## Building a neural network model with keras

In [117]:
#Load the predictor and target variables
X = cleaned_data.drop(columns=['Machine failure'])
y = cleaned_data['Machine failure']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize (scale) the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE resampling technique
smote = SMOTE()
X_train_scaled_and_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Create a neural network model
model = Sequential()
model.add(Dense(32, input_dim=X_train_scaled_and_resampled.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled_and_resampled, y_train_resampled, epochs=500, batch_size=32)

# Predict on the test data
y_pred = model.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate accuracy and recall
accuracy = accuracy_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")

Epoch 1/500
484/484 [==============================] - 0s 426us/step - loss: 0.3899 - accuracy: 0.8233
Epoch 2/500
484/484 [==============================] - 0s 430us/step - loss: 0.2199 - accuracy: 0.9117
Epoch 3/500
484/484 [==============================] - 0s 428us/step - loss: 0.1804 - accuracy: 0.9297
Epoch 4/500
484/484 [==============================] - 0s 496us/step - loss: 0.1571 - accuracy: 0.9400
Epoch 5/500
484/484 [==============================] - 0s 471us/step - loss: 0.1402 - accuracy: 0.9460
Epoch 6/500
484/484 [==============================] - 0s 450us/step - loss: 0.1282 - accuracy: 0.95250s - loss: 0.1296 - accuracy
Epoch 7/500
484/484 [==============================] - 0s 436us/step - loss: 0.1199 - accuracy: 0.9538
Epoch 8/500
484/484 [==============================] - 0s 430us/step - loss: 0.1126 - accuracy: 0.9577
Epoch 9/500
484/484 [==============================] - 0s 428us/step - loss: 0.1081 - accuracy: 0.9589
Epoch 10/500
484/484 [=======================

484/484 [==============================] - 0s 444us/step - loss: 0.0565 - accuracy: 0.9812
Epoch 80/500
484/484 [==============================] - 0s 467us/step - loss: 0.0571 - accuracy: 0.9809
Epoch 81/500
484/484 [==============================] - 0s 477us/step - loss: 0.0559 - accuracy: 0.9805
Epoch 82/500
484/484 [==============================] - 0s 461us/step - loss: 0.0565 - accuracy: 0.9805
Epoch 83/500
484/484 [==============================] - 0s 450us/step - loss: 0.0564 - accuracy: 0.9803
Epoch 84/500
484/484 [==============================] - 0s 442us/step - loss: 0.0544 - accuracy: 0.9820
Epoch 85/500
484/484 [==============================] - 0s 442us/step - loss: 0.0552 - accuracy: 0.9817
Epoch 86/500
484/484 [==============================] - 0s 436us/step - loss: 0.0551 - accuracy: 0.9817
Epoch 87/500
484/484 [==============================] - 0s 446us/step - loss: 0.0540 - accuracy: 0.9820
Epoch 88/500
484/484 [==============================] - 0s 440us/step - loss:

484/484 [==============================] - 0s 438us/step - loss: 0.0419 - accuracy: 0.9860
Epoch 158/500
484/484 [==============================] - 0s 436us/step - loss: 0.0406 - accuracy: 0.9872
Epoch 159/500
484/484 [==============================] - 0s 438us/step - loss: 0.0416 - accuracy: 0.9862
Epoch 160/500
484/484 [==============================] - 0s 450us/step - loss: 0.0417 - accuracy: 0.9858
Epoch 161/500
484/484 [==============================] - 0s 450us/step - loss: 0.0401 - accuracy: 0.9873
Epoch 162/500
484/484 [==============================] - 0s 448us/step - loss: 0.0423 - accuracy: 0.9862
Epoch 163/500
484/484 [==============================] - 0s 440us/step - loss: 0.0412 - accuracy: 0.9862
Epoch 164/500
484/484 [==============================] - 0s 449us/step - loss: 0.0399 - accuracy: 0.9871
Epoch 165/500
484/484 [==============================] - 0s 446us/step - loss: 0.0407 - accuracy: 0.9869
Epoch 166/500
484/484 [==============================] - 0s 444us/ste

484/484 [==============================] - 0s 442us/step - loss: 0.0364 - accuracy: 0.9884
Epoch 235/500
484/484 [==============================] - 0s 444us/step - loss: 0.0360 - accuracy: 0.9884
Epoch 236/500
484/484 [==============================] - 0s 440us/step - loss: 0.0358 - accuracy: 0.9891
Epoch 237/500
484/484 [==============================] - 0s 448us/step - loss: 0.0352 - accuracy: 0.9875
Epoch 238/500
484/484 [==============================] - 0s 442us/step - loss: 0.0353 - accuracy: 0.9882
Epoch 239/500
484/484 [==============================] - 0s 446us/step - loss: 0.0357 - accuracy: 0.9891
Epoch 240/500
484/484 [==============================] - 0s 446us/step - loss: 0.0346 - accuracy: 0.9896
Epoch 241/500
484/484 [==============================] - 0s 442us/step - loss: 0.0367 - accuracy: 0.9874
Epoch 242/500
484/484 [==============================] - 0s 452us/step - loss: 0.0358 - accuracy: 0.9891
Epoch 243/500
484/484 [==============================] - 0s 442us/ste

484/484 [==============================] - 0s 589us/step - loss: 0.0323 - accuracy: 0.9894
Epoch 312/500
484/484 [==============================] - 0s 552us/step - loss: 0.0314 - accuracy: 0.9908
Epoch 313/500
484/484 [==============================] - 0s 459us/step - loss: 0.0312 - accuracy: 0.9902
Epoch 314/500
484/484 [==============================] - 0s 442us/step - loss: 0.0315 - accuracy: 0.9898
Epoch 315/500
484/484 [==============================] - 0s 440us/step - loss: 0.0312 - accuracy: 0.9897
Epoch 316/500
484/484 [==============================] - 0s 442us/step - loss: 0.0346 - accuracy: 0.9895
Epoch 317/500
484/484 [==============================] - 0s 440us/step - loss: 0.0325 - accuracy: 0.9885
Epoch 318/500
484/484 [==============================] - 0s 450us/step - loss: 0.0320 - accuracy: 0.9896
Epoch 319/500
484/484 [==============================] - 0s 440us/step - loss: 0.0328 - accuracy: 0.9893
Epoch 320/500
484/484 [==============================] - 0s 448us/ste

484/484 [==============================] - 0s 442us/step - loss: 0.0291 - accuracy: 0.9902
Epoch 390/500
484/484 [==============================] - 0s 446us/step - loss: 0.0286 - accuracy: 0.9910
Epoch 391/500
484/484 [==============================] - 0s 444us/step - loss: 0.0309 - accuracy: 0.9898
Epoch 392/500
484/484 [==============================] - 0s 438us/step - loss: 0.0307 - accuracy: 0.9909
Epoch 393/500
484/484 [==============================] - 0s 440us/step - loss: 0.0279 - accuracy: 0.9911
Epoch 394/500
484/484 [==============================] - 0s 452us/step - loss: 0.0280 - accuracy: 0.9915
Epoch 395/500
484/484 [==============================] - 0s 455us/step - loss: 0.0288 - accuracy: 0.9908
Epoch 396/500
484/484 [==============================] - 0s 442us/step - loss: 0.0281 - accuracy: 0.9914
Epoch 397/500
484/484 [==============================] - 0s 440us/step - loss: 0.0291 - accuracy: 0.9909
Epoch 398/500
484/484 [==============================] - 0s 440us/ste

484/484 [==============================] - 0s 488us/step - loss: 0.0259 - accuracy: 0.9916
Epoch 468/500
484/484 [==============================] - 0s 440us/step - loss: 0.0265 - accuracy: 0.9915
Epoch 469/500
484/484 [==============================] - 0s 434us/step - loss: 0.0254 - accuracy: 0.9921
Epoch 470/500
484/484 [==============================] - 0s 438us/step - loss: 0.0274 - accuracy: 0.9906
Epoch 471/500
484/484 [==============================] - 0s 440us/step - loss: 0.0255 - accuracy: 0.9918
Epoch 472/500
484/484 [==============================] - 0s 444us/step - loss: 0.0267 - accuracy: 0.9923
Epoch 473/500
484/484 [==============================] - 0s 440us/step - loss: 0.0263 - accuracy: 0.9913
Epoch 474/500
484/484 [==============================] - 0s 444us/step - loss: 0.0251 - accuracy: 0.9919
Epoch 475/500
484/484 [==============================] - 0s 436us/step - loss: 0.0247 - accuracy: 0.9927
Epoch 476/500
484/484 [==============================] - 0s 448us/ste

## Building a MLP Classifier Neural Network with sklearn

In [118]:
X = cleaned_data.drop(columns=['Machine failure'])
y = cleaned_data['Machine failure']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE resampling technique
smote = SMOTE()
X_train_scaled_and_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Create and train a neural network classifier
clf = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42)
clf.fit(X_train_scaled_and_resampled, y_train_resampled)

# Make predictions on the test data
y_pred = clf.predict(X_test_scaled)

# Calculate accuracy and recall
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")

Accuracy: 0.9680
Recall: 0.7778
